# Naive vs. Covariance Method 

There is an interesting question of when naive method is faster than covariance method.
Due to the memory overhead that covariance method incurs, 
if we resort to some automatic method of detecting which method to use,
we must cap size of $p$ for which the covariance method is taken.
However, depending on the relationship between $n$ and $p$,
naive method may still be optimal even if $n < p < C$ where $C$ is the cap value.
This notebook attempts to use simple machine learning
to find a good heuristic for when we should use naive vs. covariance method.

In [4]:
%load_ext autoreload
%autoreload 2

In [106]:
import numpy as np
import adelie as gl
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

In [38]:
def sample_outcome(input, seed=0):
    """
    input:  (J, 3) array with row = (n, p, n_groups) configuration.
    """
    output = np.empty((input.shape[0], 2))
    for i in tqdm(range(input.shape[0])):
        n, p, n_groups = input[i]
        np.random.seed(seed)
        X, beta, y, groups, group_sizes = gl.generate_group_elnet_data(
            n, p, n_groups, rho=0.0, svd_transform=False, group_split_type="even",
        ).values()

        start = time.perf_counter() 
        out_cov = gl.group_basil(
            X, y, groups, group_sizes,
            method='cov',
        )
        end = time.perf_counter()
        cov_time = end - start

        start = time.perf_counter() 
        out_naive = gl.group_basil(
            X, y, groups, group_sizes,
            method='naive',
        )
        end = time.perf_counter()
        naive_time = end - start
        output[i] = [naive_time, cov_time]
        
    return output

In [45]:
def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

In [53]:
ns = np.arange(100, 1000 + 100, step=100)
ps = np.arange(10, 10000 + 10, step=500)
gs = np.array([5, 10])
data = cartesian_product(ns, ps, gs)

In [54]:
outcomes = sample_outcome(data)

100%|██████████| 400/400 [18:24<00:00,  2.76s/it]


In [56]:
hash_id = hash(outcomes.data.tobytes())
np.save(f"data/input_{hash_id}.csv", data)
np.save(f'data/times_{hash_id}.csv', outcomes)

In [115]:
X = data
y = outcomes[:, 0] > outcomes[:, 1]

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
print(X_train.shape)
print(X_test.shape)

(320, 3)
(80, 3)


In [107]:
lrcv = LogisticRegressionCV(
    Cs=5,
    fit_intercept=False,
    n_jobs=-1,
)
lrcv.fit(X_train, y_train)

LogisticRegressionCV(Cs=5, fit_intercept=False, n_jobs=-1)

In [123]:
probs = lrcv.predict_proba(X_test)
y_hat = probs[:, 0] <= probs[:, 1]
np.mean(y_hat != y_test)

0.05

In [164]:
my_test = np.array([
    [10, 100, 10],
    [1000, 100, 10],
    [10000, 10, 10],
])
probs = lrcv.predict_proba(my_test)
y_hat = probs[:, 0] <= probs[:, 1]
y_hat

array([False,  True,  True])

In [183]:
sample_outcome(np.array([[10, 100, 10]]))

100%|██████████| 1/1 [00:00<00:00, 110.62it/s]


array([[0.00086634, 0.00755115]])